In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('csv_files/sap_produkcyjne_przyklad.csv', encoding='utf-8-sig', sep=";")

In [11]:
with open('csv_files/sap_produkcyjne_przyklad.csv', encoding="utf-8") as f:
    first_line = f.readline()

import re
od_match = re.search(r"Od\s+(\d{2}\.\d{2}\.\d{4})", first_line)
do_match = re.search(r"Do\s+(\d{2}\.\d{2}\.\d{4})", first_line)

od_date = pd.to_datetime(od_match.group(1), dayfirst=True).date() if od_match else None
do_date = pd.to_datetime(do_match.group(1), dayfirst=True).date() if do_match else None

In [12]:
header_row_1 = df.iloc[2].fillna("").astype(str).str.strip()
header_row_2 = df.iloc[3].fillna("").astype(str).str.strip()
combined_headers = [
    (a + " " + b).strip().replace(".", "").replace("/", "").replace(" ", "")
    for a, b in zip(header_row_1, header_row_2)
]

df = df.iloc[5:].copy()
df.columns = combined_headers
df.dropna(how="all", inplace=True)
df.reset_index(drop=True, inplace=True)

In [13]:
df["Od"] = od_date
df["Do"] = do_date

In [14]:
df.columns = [col.replace('%', 'Procent') for col in df.columns]
print(df.columns)

Index(['Linia', 'Rodzina', 'QLTOTAkt', 'QLTOTPln', 'ProcentDvtProduk',
       'ZmianaCzysty', 'ZmianaPrg', 'ZmianaStd', 'QZmStandard', 'QZmStd',
       'QZmAkt', 'QZmDocel', 'QZmStd', 'QCPKAkt', 'QCPKDocel', 'QCPKStd',
       'OpeLNShAkt', 'OpeLNShDocel', 'OpeLNShStd', 'OpeELShAkt',
       'OpeELShDocel', 'OpeELShStd', 'GQLAkt', 'GQLDocel', 'GQLStd',
       'ProcentSCEff', 'ProcentSCStd', 'ProcentSCStd', 'ProcentSREff',
       'ProcentSRStd', 'ProcentSRStd', 'ProcentSFSPEff', 'ProcentSFSPStd',
       'ProcentSFSPStd', 'GodzPracAkt', 'GodzPracDocel', 'GodzPracStd',
       'ProcentELINIAEff', 'ProcentELINIAObb', 'ProcentELINIAStd',
       'ProcentEPracEff', 'ProcentEPracObb', 'ProcentEPracStd',
       'ProcentZyskuEff', 'ProcentZyskuObb', 'ProcentZyskuStd', 'Od', 'Do'],
      dtype='object')


In [15]:
float_columns = [
    "QLTOTAkt", "QLTOTPln", "ProcentDvtProduk", "ZmianaCzysty", "ZmianaPrg", "ZmianaStd",
    "QZmAkt", "QZmDocel", "QZmStd", "QCPKAkt", "QCPKDocel", "QCPKStd",
    "OpeLNShAkt", "OpeLNShDocel", "OpeLNShStd", "OpeELShAkt", "OpeELShDocel", "OpeELShStd",
    "GQLAkt", "GQLDocel", "GQLStd", "ProcentSCEff", "ProcentSCStd", "ProcentSREff", "ProcentSRStd",
    "ProcentSFSPEff", "ProcentSFSPStd", "GodzPracAkt", "GodzPracDocel", "GodzPracStd",
    "ProcentELINIAEff", "ProcentELINIAObb", "ProcentELINIAStd",
    "ProcentEPracEff", "ProcentEPracObb", "ProcentEPracStd",
    "ProcentZyskuEff", "ProcentZyskuObb", "ProcentZyskuStd"
]

df = df.loc[:, ~df.columns.duplicated()]
df.drop(columns=["QZmStandard"], inplace=True)
for col in float_columns:
    # Access column safely as a Series
    df.loc[:, col] = df[col].apply(lambda x: str(x).replace(' ', '').replace(',', '.'))
    df.loc[:, col] = pd.to_numeric(df[col], errors='coerce')

In [16]:
df = df.astype({
    "Od": "datetime64[ns]",
    "Do": "datetime64[ns]",
    "Linia": "int",
    "Rodzina": "string",
    "QLTOTAkt": "float",
    "QLTOTPln": "float",
    "ProcentDvtProduk": "float",
    "ZmianaCzysty": "float",
    "ZmianaPrg": "float",
    "ZmianaStd": "float",
    "QZmAkt": "float",
    "QZmDocel": "float",
    "QZmStd": "float",
    "QCPKAkt": "float",
    "QCPKDocel": "float",
    "QCPKStd": "float",
    "OpeLNShAkt": "float",
    "OpeLNShDocel": "float",
    "OpeLNShStd": "float",
    "OpeELShAkt": "float",
    "OpeELShDocel": "float",
    "OpeELShStd": "float",
    "GQLAkt": "float",
    "GQLDocel": "float",
    "GQLStd": "float",
    "ProcentSCEff": "float",
    "ProcentSCStd": "float",
    "ProcentSREff": "float",
    "ProcentSRStd": "float",
    "ProcentSFSPEff": "float",
    "ProcentSFSPStd": "float",
    "GodzPracAkt": "float",
    "GodzPracDocel": "float",
    "GodzPracStd": "float",
    "ProcentELINIAEff": "float",
    "ProcentELINIAObb": "float",
    "ProcentELINIAStd": "float",
    "ProcentEPracEff": "float",
    "ProcentEPracObb": "float",
    "ProcentEPracStd": "float",
    "ProcentZyskuEff": "float",
    "ProcentZyskuObb": "float",
    "ProcentZyskuStd": "float"
})

df_od = df.pop("Od")
df_do = df.pop("Do")

df.insert(3, "Od", df_od)
df.insert(4, "Do", df_do)

df

,Linia,Rodzina,QLTOTAkt,Od,Do,QLTOTPln,ProcentDvtProduk,ZmianaCzysty,ZmianaPrg,ZmianaStd,...,GodzPracStd,ProcentELINIAEff,ProcentELINIAObb,ProcentELINIAStd,ProcentEPracEff,ProcentEPracObb,ProcentEPracStd,ProcentZyskuEff,ProcentZyskuObb,ProcentZyskuStd
0,10801,Q101,1782.169,2025-06-09,2025-06-15,1780.745,0.08,19.122,19.107,19.237,...,7111.543,84.999,84.999,84.429,88.553,85.0,84.43,84.999,85.0,84.425


In [17]:
# from sqlalchemy import create_engine

# # Connection details
# server = 'JASNACZERN'        
# database = 'ferrero'
# username = 'sa'
# password = '1234'
# driver = 'ODBC Driver 17 for SQL Server'  

# # SQLAlchemy connection string
# conn_str = (
#     f"mssql+pyodbc://{username}:{password}@{server}/{database}"
#     f"?driver={driver.replace(' ', '+')}"
# )

# # Create SQLAlchemy engine
# engine = create_engine(conn_str)

# # Write the DataFrame to SQL Server (replace table_name with your desired name)
# df.to_sql('BilansProdukcji', con=engine, if_exists='append', index=False)